In [11]:
%load_ext autoreload
%autoreload 2

import cv2
import torch
from PIL import Image
from torchvision import transforms
import imgproc
from torchvision.transforms.functional import InterpolationMode as IMode
from Model.resnet import MyGenerator
import torch.nn.functional as F

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
image_dir = './jupyter_src/pic/0822.png'
image = Image.open(image_dir)
# hr_transforms = transforms.CenterCrop(96)
hr_image = image
lr_transforms = transforms.Resize([hr_image.size[1]//4, hr_image.size[0]//4] , interpolation=IMode.BICUBIC)

print(hr_image.size)
hr_image.save('./jupyter_res/pic/origin.png')
hr_image.show('Original')
lr_image = lr_transforms(hr_image)
lr_image.show("BICUBIC")
lr_image.save('./jupyter_res/pic/bicubic.png')

(2040, 1596)



(eog:283979): EOG-CRITICAL **: 15:23:15.449: eog_image_get_file: assertion 'EOG_IS_IMAGE (img)' failed

(eog:283979): GLib-GIO-CRITICAL **: 15:23:15.449: g_file_equal: assertion 'G_IS_FILE (file1)' failed


In [13]:
model = MyGenerator()
checkpoint = torch.load('./jupyter_pth/my_down_up/g-last.pth', map_location='cuda:0')
checkpoint = {k.replace('module.',''):v for k,v in checkpoint.items()}
# model = torch.nn.DataParallel(model)
model.load_state_dict(checkpoint)
model.eval()

MyGenerator(
  (conv_block0): Sequential(
    (0): Conv2d(3, 16, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
    (1): PReLU(num_parameters=1)
  )
  (down_trunk): Sequential(
    (0): ResidualConvBlock(
      (rcb): Sequential(
        (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): PReLU(num_parameters=1)
        (3): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): ResidualConvBlock(
      (rcb): Sequential(
        (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): PReLU(num_parameters=1)
        (3): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1

In [14]:
hr_tensor = imgproc.image2tensor(hr_image, range_norm=False, half=False)
lr_tensor = imgproc.image2tensor(lr_image, range_norm=False, half=False)
print(hr_tensor.shape)
with torch.no_grad():
    hr_tensor = hr_tensor.reshape(1,3,1596,2040)
    lr_tensor = lr_tensor.reshape(1,lr_tensor.shape[0], lr_tensor.shape[1], lr_tensor.shape[2])
    down = model.downsample(hr_tensor)
    reveal = model.forward(hr_tensor)
    down_img = Image.fromarray(imgproc.tensor2image(F.normalize(-down, p=2, dim=1) , range_norm=False, half=False))
    reveal_img = Image.fromarray(imgproc.tensor2image(reveal, range_norm=False, half=False))
    down_img.show('down_result')
    reveal_img.show('reveal_result')
    down_img.save('./jupyter_res/pic/feature.png')
    reveal_img.save('./jupyter_res/pic/reveal.png')

torch.Size([3, 1596, 2040])


FileNotFoundError: [Errno 2] No such file or directory: '../pic/feature.png'